Install Packages

In [1]:
!pip install bertopic

Importing Packages

In [2]:
import psycopg2
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bertopic import BERTopic

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Connecting with DB and running Query to get final data

In [3]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='cpdbstudent', password='DataSci4AI', host='codd01.research.northwestern.edu', port= '5432'
)

In [4]:
cursor = conn.cursor()

In [5]:
query = '''
select distinct a.id, a.rank, c.allegation_id, c.allegation_text
   	from data_officer a
join (
SELECT rank, sum(total_allegations) as Total_allegations
       from (select a.id, a.first_name, a.last_name, a.rank, count(b.officer_id) as total_allegations
   	from data_officer a
join data_officerallegation b
on a.id = b.officer_id
group by a.id
order by count(b.officer_id) desc)A
group by rank
order by total_allegations desc limit 10) b
on a.rank = b.rank
join (
    select id, allegation_id, allegation_text from (
        select a.id, b.allegation_id, c.cr_text as allegation_text
   	from data_officer a
join data_officerallegation b
on a.id = b.officer_id
    join data_allegation c
        on b.allegation_id = c.crid) Z
        )c
on a.id = c.id
where c.allegation_text is not null
order by a.id;
'''

In [6]:
cursor.execute(query)
data = cursor.fetchall()
print("shape is: " + str(len(data)))

df = pd.DataFrame(data)
colnames = [desc[0] for desc in cursor.description]
df.columns = colnames

shape is: 6167


In [7]:
df.head()

,id,rank,allegation_id,allegation_text
0,2,Police Officer as Detective,1049623,Initial / Intake Allegation 1: The complainant...
1,2,Police Officer as Detective,1049701,Initial / Intake Allegation 1: The reporting p...
2,16,Sergeant of Police,1049425,Initial / Intake Allegation 1: The complainant...
3,16,Sergeant of Police,1055372,Initial / Intake Allegation 1: The reporting p...
4,34,Police Officer,1055105,Initial / Intake Allegation 1: The complainant...


Split dataset to train and test


In [16]:
df_1 = df[0:6157].copy()
df_test = df[6157:].copy().reset_index(drop=True)

In [ ]:
df_1['allegation_text'][90]

'Initial / Intake Allegation 1: report. However, the\ntake fingerprints of the\nInitial / Intake Allegation 4: ‘The reporting party alleged that unknown\nofficers broke into his aparment and the\naccused filed a police report. However, the\nofficers refused to take fingerprints of the\nfront door.\nCASE REPORT\nFinding 1: NON-CPD\n'

Clean allegations text

In [18]:
def clean_text(text, remove_stopwords=True):
  ps = PorterStemmer()
  text = text.split(':')[1]
  text = re.sub('\n',' ', text)
  text = re.sub('Initial / Intake Allegation','', text)
  text = re.sub("\d+", "", text)
  text = re.sub(r'([^a-zA-Z ]+?)', "", text)
  
  # Remove stop words
  if remove_stopwords:
      stops = set(stopwords.words("english"))
      words = [w for w in text.split() if not w in stops]
      
      text_out = " ".join(words)
  
  text_out = " ".join([ps.stem(i) for i in text_out.split()])
  text_out = text_out.lower()
  return text_out

In [ ]:
final_text = []
for i in range(len(df_1)):
  text = clean_text(df_1['allegation_text'][i])
  final_text.append(text)

In [ ]:
final_text[0:5]

['the complain wit incid alleg accus offic kick taser son handcuf without justif',
 'the report parti alleg friend sit vehicl eat wait anoth friend come currenc exchang accus approach vehicl remov everyon vehicl search without justif the report parti also alleg name accus fals told warrant report parti deni howev accus took th district station anyway the report parti alleg district station strip search forc bend open butt cheek rais genit without justif the report parti alleg accus state give someth name hous someth inform accus inform give accus fals arrest the report parti state everi time accus see harass without justif',
 'the complain alleg accus offic detain handcuf anoth male subject without justif the complain alleg accus offic threaten plant drug weapon order catch case the complain alleg racial profil find',
 'the report parti alleg unknown hispan male unform offic fail file report regard lawn bumt due firework land lawn',
 'the complain alleg accus offic rude yell use profan

In [ ]:
final_text[0]

'the complain wit incid alleg accus offic kick taser son handcuf without justif'

Topic modeling using BERTopic

In [ ]:
topic_model = BERTopic()

In [ ]:
topics, _ = topic_model.fit_transform(final_text)

In [ ]:
df_1['topics_1'] = topics
df_1[df_1['topics_1'] == 4]['allegation_text']

275     Initial / Intake Allegation 1: The reporting p...
357     Initial / Intake Allegation 1: The reporting p...
402     Initial / Intake Allegation 1: It is alleged b...
519     Initial / Intake Allegation 1: The reporting p...
526     Initial / Intake Allegation 1: The complainant...
637     Initial / Intake Allegation 1: his landlord's ...
638     Initial / Intake Allegation 1: The reporting p...
675     Initial / Intake Allegation 1: The complainant...
681     Initial / Intake Allegation 1: The reporting p...
720     Initial / Intake Allegation 1: The complainant...
812     Initial / Intake Allegation 1: The reporting p...
915     Initial / Intake Allegation 1: The reporting p...
1136    Initial / Intake Allegation 1: The reporting p...
1275    Initial / Intake Allegation 1: The complainant...
1421    Initial / Intake Allegation 1: The reporting p...
1549    Initial / Intake Allegation 1: The reporting p...
1583    Initial / Intake Allegation 1: The complainant...
1594    Initia

In [ ]:
df_1[df_1['topics_1'] == 4]['allegation_text'][6129]

'Initial / Intake Allegation 1: The reporting party alleges that she did\nnot receive police assistance regarding a\ntenant and landlord dispute and stolen\nproperty\nInitial / Intake Allegation 4: The reporting party alleges that she did\nnot receive police assistance regarding a\ntenant and landlord dispute and stolen\nproperty.\nFinding 1: (None Entered)\n'

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1325,-1_falsifi_retaili_chicago_broke
1,0,82,0_rude_disrespect_speak_shirt
2,1,62,1_smoke_cigarett_harm_cta
3,2,62,2_accid_ticket_traffic_tegard
4,3,60,3_usc_overag_discrep_cashier
...,...,...,...
204,203,11,203_ipra_burglar_password_videotap
205,204,11,204_did_wit_minor_twice
206,205,11,205_abduct_mother_daughter_dog
207,206,11,206_repli_stroller_homeless_restaur


In [ ]:
topic_model.get_topic(4)

[('landlord', 0.059059504118049444),
 ('tenant', 0.03651305871908606),
 ('grow', 0.01875900169555575),
 ('basement', 0.018139521480176345),
 ('rent', 0.013702569929593355),
 ('build', 0.012414010401981115),
 ('evelyn', 0.010118972508132713),
 ('evict', 0.009241354960761106),
 ('henderson', 0.0072128384459608594),
 ('landlordten', 0.0072128384459608594)]

In [ ]:
topic_model.visualize_topics()

Reduce number of topics to cluster similar topics 

In [ ]:
topic_model = BERTopic(nr_topics=6) #similar topics will be merged

In [ ]:
topics, _ = topic_model.fit_transform(final_text)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1419,-1_report_parti_victim_polic
1,0,1248,0_alleg_accus_victim_complain
2,1,1111,1_report_parti_accus_state
3,2,859,2_offic_report_door_parti
4,3,587,3_report_parti_sergeant_arrest
5,4,468,4_vehicl_search_arrest_victim
6,5,465,5_parti_alleg_inventori_plant


In [ ]:
df_1['topics_2'] = topics
df_1[df_1['topics_2'] == 4]['allegation_text']

11      Initial / Intake Allegation 1: The complainant...
32      Initial / Intake Allegation 1: report, and tol...
74      Initial / Intake Allegation 1: The reporting p...
86      Initial / Intake Allegation 1: The plaintiff a...
121     Initial / Intake Allegation 1: The complainant...
                              ...                        
6144    Initial / Intake Allegation 1:  The reporting ...
6146    Initial / Intake Allegation 1: The reporting p...
6147    Initial / Intake Allegation 1: The complainant...
6151    Initial / Intake Allegation 1: The complainant...
6156    Initial / Intake Allegation 1: to search her r...
Name: allegation_text, Length: 468, dtype: object

In [ ]:
df_1[df_1['topics_2'] == 4]['allegation_text'][6151]

'Initial / Intake Allegation 1: The complainant alleges that the accused\nofficer searched his vehicle without\njustification\nThe complainant alleges that the accused\nofficer detained him without justification.\nFinding 1: (None Entered)\n'

In [ ]:
final_text[6151]

'the complain alleg accus offic search vehicl without justif the complain alleg accus offic detain without justif find'

In [ ]:
topic_model.get_topic(4)

[('vehicl', 0.06328782794183578),
 ('search', 0.05911232458968046),
 ('arrest', 0.05799489396934193),
 ('victim', 0.04596237899730433),
 ('warrant', 0.02818013678229811),
 ('coerc', 0.028153415677698215),
 ('impound', 0.023704699922724427),
 ('permiss', 0.02341500840813096),
 ('detain', 0.023018069050091985),
 ('plaintiff', 0.019480062816899735)]

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_heatmap()

Saving model created for topic clustering


In [ ]:
topic_model.save("topics_model")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



Topic clustering on test data with model created on train data

In [14]:
BerTopic_model = BERTopic.load("topics_model")

In [19]:
test_text = []
for i in range(len(df_test)):
  text = clean_text(df_test['allegation_text'][i])
  test_text.append(text)

In [21]:
topics, probs = BerTopic_model.transform(test_text)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [22]:
df_test['final_topic'] = topics
df_test

,id,rank,allegation_id,allegation_text,final_topic
0,32424,Police Officer,1056276,Initial / Intake Allegation 1: The reporting p...,1
1,32430,Police Officer,1055255,Initial / Intake Allegation 1: The complainant...,-1
2,32430,Police Officer,1058438,Initial / Intake Allegation 1: THE REPORTING ...,0
3,32430,Police Officer,1058984,Initial / Intake Allegation 1: a warrant and\n...,2
4,32432,Sergeant of Police,1075034,Initial / Intake Allegation 1: THE REPORTING P...,3
5,32433,Police Officer Assigned Evidence Technician,1060383,Initial / Intake Allegation 3: The reporting p...,5
6,32435,Police Officer,1049816,Initial / Intake Allegation 1: The reporting p...,-1
7,32438,Police Officer,1057765,Initial / Intake Allegation 1: The reporting p...,2
8,32442,Police Officer,1057569,Initial / Intake Allegation 1: The reporting p...,0
9,32465,Police Officer,1052346,Initial / Intake Allegation 1: THE REPORTING P...,1
